# Домашнее задание для Junior DS по разработке PD модели

<b>Задание:</b> Необходиом разработать модель предсказания дефолта. Даны финансовые данные 32 395 компаний (выручка, активы, пассивы и т.д). Результат будет оцениваться на скрытой выборке из 200 компаний по метрике качества accuracy_score

<b>Цель задания</b> научиться / показать знания в разработке моделей оценки риска - probability of default. Задание приближено к реальному, данные синтетические. Лидерборд для оценки собственного решения и геймификации процесса обучения. Первое место в лидерборде не явлется целью. Цель - правильно разарботанная модель, удовлетворяющая требованиям банковской системы. В случае возникновения вопросов, сотрудники блока Риски готовы проконсультировать и дать обратную связь на модель (телеграм чат).

<b>Метрика качества</b> accuracy_score вычисляется, как ( True positive + True negative ) / Total. Если сабмит-файл, не правильного формата или неверное количество строк, метрика будет = 0.

### Импортируем библиотеки

In [252]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

### Подгружаем данные

In [253]:
train_df = pd.read_csv('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv', sep=';')
test_df = pd.read_csv('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv', sep=';')
desc_df = pd.read_csv('https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv', sep=';')

In [254]:
train_df.head(10)

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,0.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,0.0,198,136,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256,135,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,33,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,0.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,0.0,185,139,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,0.0,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,0.0,220,138,27,27,27,[1-100],10000.0,2,0,0,0,0
5,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,100,5,5,5,[1-100],350000.0,1,0,0,0,0
6,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266,27,7,7,7,[1-100],35000.0,3,0,0,0,0
7,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,31,30,30,30,[1-100],10000.0,2,0,0,0,0
8,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,27,26,26,26,[1-100],10000.0,1,0,0,0,0
9,40,125432000.0,122655000.0,122655000.0,0.0,0.0,2777000.0,-63000.0,2714000.0,579000.0,0.0,2135000.0,0.0,24680000.0,8263000.0,16417000.0,0.0,106000.0,2135000.0,11267000.0,11912000.0,0.0,13000.0,11899000.0,0.0,170,132,15,15,15,(100-500],10000.0,2,0,0,0,0


Описание данных

In [255]:
desc_df.head(50)

,field,desc_eng,desc_rus
0,ar_revenue,Sales revenue (at the end of the last year),Выручка от реализации (на конец последнего года)
1,ar_total_expenses,Total expenses for the last year,Общие расходы за последний год
2,ar_sale_cost,Cost of sales (at the end of the last year),Себестоимость продаж (на конец последнего года)
3,ar_selling_expenses,Selling expenses (at the end of the last year),Коммерческие расходы (на конец последнего года)
4,ar_management_expenses,Administrative expenses (at the end of the las...,Управленческие расходы (на конец последнего года)
5,ar_sale_profit,Profit from sales (at the end of the last year),Прибыль от реализации (на конец последнего года)
6,ar_balance_of_rvns_and_expns,Balance of income and expenses (at the end of ...,Баланс доходов и расходов (на конец последнего...
7,ar_profit_before_tax,Gross Profit (end of last year),Валовая прибыль (на конец последнего года)
8,ar_taxes,Current income tax (at the end of the last year),Текущий на налог на прибыль (на конец последне...
9,ar_other_profit_and_losses,Other income and expenses (at the end of the l...,Прочие доходы и расходы (на конец последнего г...


Чтобы разобраться в признаках, давайте углубимся в предметную область. В задании сказано, что мы работаем не с ИП, а с ООО, поэтому будем обращать особое внимание на порядок оформления ООО. Для начала участники ООО названчают руководителя и вместе с этим устанавливает капитал и регистрируют юридический адрес,тот адрес, по которому ведомства будут связываться с компанией: присылать запросы, письма, требования и уведомления. Затем необходимо выбрать коды видов деятельности: название и цифровой код для дальнейшего обращения в налоговую службу. До регистрации ООО нужно определиться с системой налогооблежния и собрать все учредительные документы. В течение 10 дней после передачи документов в налоговую службу ООО должен быть присвоен ОГРН - основной государственный регистрационный номер. 



Исходя из вышесказанного, посмотрим на наши признаки: 

В наших данных есть признаки с разными названиями, но с одним и тем же назначением, например: 
1. adr_actual_age, head_actual_age, cap_actual_age  
При регистрации юридического адреса необходимо указать руководителя и установить капитал, поэтому все эти признаки обозначают одну и ту же величину. 

2. ogrn_age и bus_age. При присваении ОГРН владелец бизнеса может официально начать вести свою дейтельность, поэтому эти признаки обозначают одну и ту же величину, за исключением, редких случаев. 

3. ar_net_profit и ar_profit_before_tax будут определенно коррелировать. Так как они отличаются только тем, что одна прибыль с учетом налога, а другая нет. Поэтому можем удалить один из этих признаков. 
4. ar_total_expenses и ar_sale_cost также могут коррелировать, если у нас фиксированы расходы, не включающие себестоимость продаж. 

Семплы и размеры выборок

In [256]:
train_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,0.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,0.0,198,136,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256,135,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,33,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,0.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,0.0,185,139,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,0.0,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,0.0,220,138,27,27,27,[1-100],10000.0,2,0,0,0,0


In [257]:
train_df.shape

(32395, 37)

Количество разных типов фичей

In [258]:
train_df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

Уровень дефолт-рейта в выборке = 6,7219%

In [259]:
train_df['default_12m'].values.mean()

0.06454699799351751

In [260]:
train_df.record_id.nunique()

32395

32395 уникальных наблюдений, что совпадает с размерностью нашей выборки. Следовательно, нам не надо бороться с проблемой дублирования строк. И первичным ключом является record_id. Этот признак в модели нам не нужен, поэтому можем его удалить. 

In [261]:
del train_df['record_id']

In [262]:
train_df.describe()

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
count,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,32395.000000,32395.000000,32395.000000,32395.000000,32395.000000,3.239500e+04,32395.000000,32395.000000,32395.000000,32395.0,32395.000000
mean,8.153776e+07,7.678474e+07,7.320328e+07,2.499198e+06,1.082263e+06,4.753016e+06,-6.424773e+05,4.110538e+06,4.188071e+05,3.937304e+04,3.731104e+06,9.007101e+06,3.044379e+07,1.627389e+07,1.336024e+07,8.096597e+05,1.610857e+06,3.307750e+06,1.911815e+07,2.419023e+07,3.149311e+06,4.983973e+06,1.508316e+07,9.737877e+05,97.121500,78.644822,15.010341,15.010341,15.010341,1.366509e+06,11.946535,0.021670,0.000093,0.0,0.064547
std,1.274402e+08,1.227227e+08,1.156651e+08,1.231798e+07,7.848929e+06,1.941027e+07,6.014715e+06,1.938021e+07,1.565298e+06,4.843389e+05,1.902355e+07,3.642723e+07,6.331020e+07,3.729649e+07,3.851823e+07,8.222404e+06,1.067265e+07,8.895007e+06,4.603456e+07,5.917809e+07,2.764231e+07,1.359236e+07,3.793929e+07,1.493730e+07,75.555398,46.522534,9.546511,9.546511,9.546511,1.502438e+07,86.341847,0.342715,0.009623,0.0,0.245729
min,0.000000e+00,-7.019840e+08,-5.072780e+08,-3.542900e+07,-1.921670e+08,-9.210000e+07,-7.581800e+07,-5.946600e+07,-1.681000e+07,-1.034000e+07,-4.778100e+07,0.000000e+00,0.000000e+00,0.000000e+00,-1.160400e+07,-3.413000e+06,0.000000e+00,-4.780300e+07,-7.501000e+07,0.000000e+00,-3.068000e+06,-2.700000e+06,0.000000e+00,-2.500000e+05,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,0.000000,0.0,0.000000
25%,1.019950e+07,9.130250e+06,8.844000e+06,0.000000e+00,0.000000e+00,2.112500e+05,-9.400000e+05,1.420000e+05,0.000000e+00,0.000000e+00,1.010000e+05,0.000000e+00,2.854500e+06,8.100000e+05,4.722500e+05,0.000000e+00,4.100000e+04,9.400000e+04,4.532500e+05,1.634000e+06,0.000000e+00,0.000000e+00,6.722500e+05,0.000000e+00,35.000000,35.000000,6.000000,6.000000,6.000000,1.000000e+04,1.000000,0.000000,0.000000,0.0,0.000000
50%,3.233500e+07,2.970350e+07,2.840800e+07,0.000000e+00,0.000000e+00,1.231500e+06,-1.260000e+05,8.500000e+05,4.100000e+04,0.000000e+00,7.040000e+05,0.000000e+00,1.109450e+07,4.714500e+06,3.028500e+06,0.000000e+00,2.640000e+05,6.730000e+05,3.528000e+06,7.231000e+06,0.000000e+00,3.610000e+05,3.678000e+06,0.000000e+00,79.000000,77.000000,14.000000,14.000000,14.000000,1.000000e+04,1.000000,0.000000,0.000000,0.0,0.000000
75%,8.887675e+07,8.309500e+07,7.991425e+07,0.000000e+00,0.000000e+00,4.574750e+06,0.000000e+00,3.543750e+06,2.910000e+05,0.000000e+00,3.176000e+06,3.490000e+05,3.423000e+07,1.754600e+07,1.200325e+07,0.000000e+00,1.188500e+06,3.025750e+06,1.760775e+07,2.541800e+07,0.000000e+00,3.816750e+06,1.424575e+07,0.000000e+00,137.000000,133.000000,24.000000,24.000000,24.000000,3.000000e+04,2.000000,0.000000,0.000000,0.0,0.000000
max,7.946840e+08,7.949360e+08,7.624930e+08,2.746560e+08,3.023550e+08,1.434511e+09,1.584250e+08,1.481526e+09,5.420200e+07,3.979500e+07,1.485559e+09,1.630344e+09,3.175805e+09,2.890917e+09,1.711586e+09,3.336220e+08,1.271337e+09,2.470140e+08,1.739879e+09,3.349389e+09,2.919676e+09,3.223740e+08,1.820521e+09,1.251578e+09,921.000000,149.000000,33.000000,33.000000,33.000000,1.584979e+09,998.000000,2

Заметим, что стандратное отклонение у некоторых признаков достаточно большое, пожтому нам нужно будет проводить стандартизацию. 

In [263]:
train_df.nunique()

ar_revenue                      12893
ar_total_expenses               12779
ar_sale_cost                    12664
ar_selling_expenses              1994
ar_management_expenses           1093
ar_sale_profit                   7163
ar_balance_of_rvns_and_expns     5063
ar_profit_before_tax             6526
ar_taxes                         2247
ar_other_profit_and_losses        595
ar_net_profit                    6231
ab_immobilized_assets            3498
ab_mobile_current_assets        11222
ab_inventory                     9143
ab_accounts_receivable           8283
ab_other_current_assets          1210
ab_cash_and_securities           3677
ab_losses                        6175
ab_own_capital                   9090
ab_borrowed_capital             10261
ab_long_term_liabilities         2294
ab_short_term_borrowing          4896
ab_accounts_payable              8797
ab_other_borrowings              1437
bus_age                           315
ogrn_age                          150
adr_actual_a

Итак, мы обнаружиди еще один бесполезный признак, у которого множество значений состоит из одного элемента, это признак 'ul_systematizing_flg', можем его удалить. 

In [264]:
del train_df['ul_systematizing_flg']

Прежде всего для выявления корреляций и замены пропусков, давайте отбросим аномалии, так как если мы это сделаем после, то мы не сможем посчитать нормальный коэффициент корреляции Пирсона (в его формуле используется среднее значение по признаку, а оно будет не смещенным при наличии выбросов).

### Поиск аномалий

Есть несколько способов для выявления и удаления выбросов, но давайте попробуем сначала самый простой. Из основных способов: 
1. Z-score, но при своем подсчете использует среднее, так что не найс. 
1. Функция Антона

In [279]:
Q1 = train_df.quantile(0.01)
Q3 = train_df.quantile(0.99)

IQR = Q3 - Q1
train_df_outliers = train_df[~((train_df < (Q1 - 1.5 * IQR)) |(train_df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [280]:
print('Размерность выборки до удаления выбросов: {}'.format(train_df.shape))
print('Размерность выборки после удаления выбросов: {}'.format(train_df_outliers.shape))

Размерность выборки до удаления выбросов: (32395, 35)
Размерность выборки после удаления выбросов: (31498, 35)


In [281]:
train_df_outliers['default_12m'].value_counts()

0    29422
1     2076
Name: default_12m, dtype: int64

### Обнаружение корреляций

Наша задача в этом разделе - обнаружить линейно-зависимые признаки и их удалить. 
Давайте добавим признаки, которые по нашему мнению, могут обозначать сумму нескольких имеющихся признаков. Например, признак 'ar_total_expenses' скорее всего содержит в себе сумму ar_sale_cost, ar_selling_expenses и ar_management_expenses. Давайте добавим сумму вышеупомянутых признаков в качестве отдельного столбца total_cost, а затем посмотрим на его корреляцию с признаком 'ar_total_expenses'.

In [282]:
train_df_outliers['total_cost'] = train_df_outliers['ar_sale_cost'] + train_df_outliers['ar_selling_expenses'] + train_df_outliers['ar_management_expenses']

Аналогичную операцию проведем и с чистой выручкой. Чистая выручка представляет из себя сумму признаков ar_profit_before_tax и ar_net_profit. 

In [283]:
train_df_outliers['net_profit'] = train_df_outliers['ar_profit_before_tax'] + train_df_outliers['ar_net_profit']

Аналогично...

In [284]:
train_df_outliers['borrowed_capital'] = train_df_outliers['ab_long_term_liabilities'] + train_df_outliers['ab_short_term_borrowing'] + train_df_outliers['ab_accounts_payable'] + train_df_outliers['ab_other_borrowings']

Аналогично...

In [285]:
train_df_outliers['revenue'] = train_df_outliers['ar_total_expenses'] + train_df_outliers['ar_sale_profit']

Аналогично....

In [286]:
train_df_outliers['mobile_current_assets'] = train_df_outliers['ab_inventory'] + train_df_outliers['ab_accounts_receivable']

In [287]:
def find_correlated_features(df_, threshold):
    corr_matrix = df_.corr().abs()
    corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))
    df_ = corr_matrix.stack().reset_index()
    df_.columns = ['feature1', 'feature2', 'corr_coef']
    df_ = df_[df_.corr_coef >= threshold].sort_values('corr_coef', ascending=False)
    return df_

In [288]:
find_correlated_features(train_df_outliers, 0.6)

,feature1,feature2,corr_coef
66,ar_total_expenses,total_cost,1.000000
621,head_actual_age,cap_actual_age,1.000000
612,adr_actual_age,cap_actual_age,1.000000
611,adr_actual_age,head_actual_age,1.000000
34,ar_revenue,revenue,1.000000
527,ab_borrowed_capital,borrowed_capital,1.000000
389,ab_mobile_current_assets,mobile_current_assets,0.998305
256,ar_profit_before_tax,net_profit,0.997970
337,ar_net_profit,net_profit,0.997567
658,total_cost,revenue,0.997491


In [289]:
del train_df_outliers['head_actual_age']
del train_df_outliers['cap_actual_age']
del train_df_outliers['bus_age']
del train_df_outliers['ab_borrowed_capital']
del train_df_outliers['ar_revenue']
del train_df_outliers['ar_total_expenses']
del train_df_outliers['ar_taxes']
del train_df_outliers['ab_mobile_current_assets']

In [290]:
# удаляем признаки, которые мы создали сами для проверки линейной зависимости
del train_df_outliers['revenue'] 
del train_df_outliers['net_profit']
del train_df_outliers['total_cost']
del train_df_outliers['mobile_current_assets']

In [291]:
del train_df_outliers['ab_losses']
del train_df_outliers['ar_profit_before_tax']
del train_df_outliers['ab_accounts_payable']
del train_df_outliers['ar_net_profit']

In [292]:
train_df_outliers.shape

(31498, 24)

### Типы признаков

In [293]:
train_df_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31498 entries, 0 to 32394
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ar_sale_cost                  16473 non-null  float64
 1   ar_selling_expenses           16473 non-null  float64
 2   ar_management_expenses        16473 non-null  float64
 3   ar_sale_profit                16473 non-null  float64
 4   ar_balance_of_rvns_and_expns  16473 non-null  float64
 5   ar_other_profit_and_losses    16473 non-null  float64
 6   ab_immobilized_assets         16473 non-null  float64
 7   ab_inventory                  16473 non-null  float64
 8   ab_accounts_receivable        16473 non-null  float64
 9   ab_other_current_assets       16473 non-null  float64
 10  ab_cash_and_securities        16473 non-null  float64
 11  ab_own_capital                16473 non-null  float64
 12  ab_long_term_liabilities      16473 non-null  float64
 13  a

Все признаки числовые, кроме одного, ul_staff_range.

In [294]:
train_df_outliers['ul_staff_range'].value_counts()

[1-100]      28657
(100-500]     2529
> 500          312
Name: ul_staff_range, dtype: int64

Этот признак можно закодировать через LabelEncoder.

In [295]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_df_outliers['ul_staff_range'] = le.fit_transform(train_df_outliers['ul_staff_range'] )

### Стандартизация

In [301]:
train_df_outliers.describe()

,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_other_profit_and_losses,ab_immobilized_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_own_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_other_borrowings,ogrn_age,adr_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,default_12m,borrowed_capital
count,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,1.647300e+04,31498.000000,31498.000000,31498.000000,3.149800e+04,31498.000000,31498.0,31498.0,31498.000000,1.647300e+04
mean,6.735824e+07,1.961989e+06,7.038106e+05,4.058385e+06,-5.882518e+05,2.364803e+04,6.516087e+06,1.424668e+07,1.099652e+07,3.256347e+05,1.301645e+06,1.601239e+07,2.170261e+06,4.322812e+06,3.419119e+05,77.620516,14.971998,1.829513,6.561572e+05,5.676360,0.0,0.0,0.065909,1.965916e+07
std,1.072794e+08,9.671108e+06,4.366645e+06,9.032176e+06,4.556331e+06,1.497990e+05,2.202381e+07,2.493200e+07,2.313252e+07,2.364454e+06,3.191715e+06,3.379051e+07,8.400831e+06,1.058665e+07,1.791719e+06,46.401355,9.529689,0.549557,3.846601e+06,37.640188,0.0,0.0,0.248127,3.389751e+07
min,-5.309500e+07,-1.406100e+07,-8.335000e+06,-3.957000e+07,-6.972900e+07,-7.690000e+05,0.000000e+00,0.000000e+00,-1.160400e+07,-3.413000e+06,0.000000e+00,-5.851700e+07,-3.068000e+06,-2.700000e+06,-2.500000e+05,0.000000,1.000000,0.000000,0.000000e+00,1.000000,0.0,0.0,0.000000,0.000000e+00
25%,8.301000e+06,0.000000e+00,0.000000e+00,2.080000e+05,-8.800000e+05,0.000000e+00,0.000000e+00,7.620000e+05,4.300000e+05,0.000000e+00,3.600000e+04,4.100000e+05,0.000000e+00,0.000000e+00,0.000000e+00,34.000000,6.000000,2.000000,1.000000e+04,1.000000,0.0,0.0,0.000000,1.503000e+06
50%,2.650600e+07,0.000000e+00,0.000000e+00,1.168000e+06,-1.220000e+05,0.000000e+00,0.000000e+00,4.367000e+06,2.794000e+06,0.000000e+00,2.440000e+05,3.102000e+06,0.000000e+00,3.140000e+05,0.000000e+00,75.000000,14.000000,2.000000,1.000000e+04,1.000000,0.0,0.0,0.000000,6.626000e+06
75%,7.334100e+07,0.000000e+00,0.000000e+00,4.242000e+06,0.000000e+00,0.000000e+00,1.530000e+05,1.611100e+07,1.081700e+07,0.000000e+00,1.061000e+06,1.505400e+07,0.000000e+00,3.493000e+06,0.000000e+00,131.000000,24.000000,2.000000,2.500000e+04,2.000000,0.0,0.0,0.000000,2.231900e+07
max,7.604440e+08,1.494320e+08,7.787000e+07,1.127170e+08,5.991900e+07,2.511000e+06,3.074030e+08,3.006660e+08,3.364400e+08,4.617300e+07,4.442300e+07,4.620080e+08,1.318680e+08,1.468000e+08,3.080800e+07,149.000000,33.000000,2.000000,6.500792e+07,816.000000,0.0,0.0,1.000000,4.505650e+08


In [302]:
train_x = train_df_outliers.drop('default_12m',axis=1)

In [304]:
scaler = StandardScaler()
scaler.fit(train_x)
train_x = pd.DataFrame(scaler.transform(train_x), index=train_x.index, columns=train_x.columns)
train_x['default_12m'] = train_df_outliers['default_12m']
train_df_std = train_x.copy()

### Обработка пропусков

In [305]:
print("\nКоличество пропущенных значений:")
train_df_std.isna().sum().to_frame().sort_values(0, ascending = False).T


Количество пропущенных значений:


,ar_sale_cost,ab_other_current_assets,borrowed_capital,ab_other_borrowings,ab_short_term_borrowing,ar_selling_expenses,ab_own_capital,ab_cash_and_securities,ab_long_term_liabilities,ab_accounts_receivable,ab_inventory,ab_immobilized_assets,ar_other_profit_and_losses,ar_balance_of_rvns_and_expns,ar_sale_profit,ar_management_expenses,ogrn_age,adr_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,default_12m
0,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,15025,0,0,0,0,0,0,0,0


Пропусков в данных сликшом много.  
1 вариант: удалить столбцы с пропусками  
2 вариант:  заменить пропуски на среднее  
3 вариант:  удалить строки с NaN
4 вариант: заполнить пропуски через KNN

In [314]:
# 1 вариант 
train_df_std_1 = train_df_std.dropna(axis = 1) 

In [311]:
# 2 вариант 
train_df_std_2 = train_df_std.fillna(train_df_std.mean())

In [317]:
# 3 вариант 
train_df_std_3 = train_df_std.dropna() 

In [325]:
# 4 вариант 
#columns_with_nan = train_df_std.columns[train_df_std.isna().any()].tolist()

In [332]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
train_df_std_4_nan = pd.DataFrame(imputer.fit_transform(train_df_std), 
                                  index=train_df_std.index, columns=train_df_std.columns)

In [330]:
# columns_without_nan = train_df_std.columns[train_df_std.notna().any()].tolist()

In [ ]:
# train_df_std_without_nan = train_df_std[columns_without_nan]

In [ ]:
# train_df_std_4 = pd.concat([])

При 1 варианте accuracy равна: 0.9301587301587302  
При 2 варианте accuracy равна: 0.930031746031746  
При 3 варианте accuracy равна: 0.9463462005341102

### Бейзлайн с этими признаками

In [318]:
train_df_std['default_12m'].value_counts()

0    29422
1     2076
Name: default_12m, dtype: int64

In [319]:
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(train_df_std_3.drop('default_12m',axis=1),
                                                  train_df_std_3['default_12m'],
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(max_iter=1000).fit(X_train, y_train)
y_pred = logit.predict(X_test)
accuracy_score(y_test, y_pred)

0.9463462005341102

In [321]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.95      1.00      0.97      3899
     class 1       0.00      0.00      0.00       220

    accuracy                           0.95      4119
   macro avg       0.47      0.50      0.49      4119
weighted avg       0.90      0.95      0.92      4119



### Начало их кода 

### Строим бэйзлайн модель c 3-ми фичами

In [7]:
features = ['bus_age','ul_capital_sum', 'ul_founders_cnt']

In [8]:
features_df = train_df.loc[:,features]
features_df['default_12m'] = train_df['default_12m']

In [9]:
features_df.head()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
0,198,50000.0,2,0
1,256,10000.0,5,0
2,33,10000.0,2,0
3,185,300000.0,1,0
4,220,10000.0,2,0


### Построим карту коллеряций

In [10]:
features_df.corr()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
bus_age,1.000000,0.066929,0.209451,-0.109389
ul_capital_sum,0.066929,1.000000,0.029165,-0.019452
ul_founders_cnt,0.209451,0.029165,1.000000,-0.023244
default_12m,-0.109389,-0.019452,-0.023244,1.000000


### Валидация результата

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features_df.drop('default_12m',axis=1),
                                                    features_df['default_12m'],
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=1000)
cv_baseline = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv_baseline

0.6483643068016243

ROC/AUC бэйзлайн модели - 64,83%

### Построение модели на всех данных

In [12]:
logit.fit(features_df.drop('default_12m',axis=1), features_df['default_12m'])

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
logit.predict(X_test, y_test)

### Формирование файла с ответом

In [ ]:
answ_df = test_df[['record_id']]

In [ ]:
answ_df['id'] = answ_df['record_id']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:

answ_df.drop('record_id',axis=1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
cutoff = y_train.mean()

In [ ]:
answ_df['predict'] = (logit.predict_proba(test_df.loc[:,features])[:,1]>cutoff).astype(int)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
answ_df.to_csv('PD-submit.csv',index=False, sep=';')

Для загрузки своего рузультата на https://dsbattle.com/hackathons/juniords-new/ вам нужно создать csv файл с двумя колонками (id, predict), разделитель - точка с запятой (;)